# Visualisation of Bessel-Gauss propagation

This notebook visualises the results of the Bessel-Gauss profiles intitiated by [this notebook](prepare_Bessel.ipynb). We select 4 of the results and animate the passage of the pulse through medium. See other notebooks ([link 1](../gas_cell/xxx), [link 2](../density_profile/analyse_density_profile.ipynb)) for different analyses.

## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
from contextlib import ExitStack
import os
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


# %%capture
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

# %matplotlib agg
%matplotlib inline

## Load data

Here we load the selected data from an input list. The provided test cases are for `30fs` and `60fs` (the durations of the initial pulses).

In [ ]:
demos_path = os.path.join(os.environ['MULTISCALE_DEMOS'],'Bessel') # path to the data, using the environment path to the demos
series_version = '30fs'                                            # '30fs' or '60fs'

h5files = [os.path.join(demos_path, series_version, foo) for foo in 
            ['results_Bessel_1.h5','results_Bessel_3.h5','results_Bessel_7.h5','results_Bessel_9.h5']] # list of loaded simulations

# # Mix the same results from different simulation series
# import itertools
# h5files = [os.path.join(demos_path, foo, bar) for foo, bar in 
#             itertools.product(['30fs','60fs'],['results_Bessel_3.h5','results_Bessel_9.h5'])]
             

# load the data
Nsim = len(h5files)
CUPRAD_res = []
with ExitStack() as stack:
    # Open all files and store file objects in a list
    files = [stack.enter_context(h5py.File(h5file, 'r')) for h5file in h5files]
    
    # Load data from each file and append it to CUPRAD_res
    for f in files:
        CUPRAD_res.append(dfC.get_data(f))


## Plot the propagating pulse
We plot the data for all the loaded simulations. We set the shared limits for the plots defined relatively to the pulse duration and set the stride (`frame_multiplier`) in $z$ for the plotting (using all the points might be resource consuming). We also choose whether to save the animation and set `fps` to ensure appealing output.

In [ ]:
tlim_relative_to_pulse_duration = (-2., 2.)
tlim = [np.asarray([t_plot_span*result.pulse_duration_entry for t_plot_span in tlim_relative_to_pulse_duration]) for result in CUPRAD_res] # [fs]

frame_multiplier = 8 # stride in z for plotting

save_animation   = True
fps              = 5 # fps to synchronise output with the frame_multiplier
animation_author = 'Jan Vábek'



Finally, we generate the figure. it allows different shapes of input simulations. However, we assume a similar spacing in $z$, and the simulation animates until the shortest $z$-axis is consumed.

In [ ]:
# Code to generate the animated figure

# find subarrays (assume general case with different input archives)
k_t_min, k_t_max = tuple(zip(*[mn.FindInterval(1e15*CUPRAD_res[k1].tgrid,1.05*tlim[k1]) for k1 in range(len(CUPRAD_res))]))


Nrows = Nsim//2 if (Nsim%2==0) else (Nsim//2)+1

fig, axes = plt.subplots(Nrows, 2, figsize=(15, Nrows*4))  # Create a 2x2 grid of subplots
axes = axes.flatten()


pcs = []; cbars = []
# Create pcolormesh plots and colorbars
for k1, ax in enumerate(axes[:len(CUPRAD_res)]):
    pc = ax.pcolormesh(1e15*CUPRAD_res[k1].tgrid[k_t_min[k1]:k_t_max[k1]],
                       1e6*CUPRAD_res[k1].rgrid,
                       1e-9*CUPRAD_res[k1].E_zrt[0, :, k_t_min[k1]:k_t_max[k1]],
                       shading='auto', cmap='seismic')
    pcs.append(pc)
    cbar = fig.colorbar(pc, ax=ax)
    cbar.ax.set_ylabel(r'$\mathcal{E}$ [GV/m]', rotation=90)
    cbars.append(cbar)
    

    # Set axis properties
    ax.set_xlim(tlim[k1])
    ax.set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[0]))
    ax.set_xlabel(r'$t~[\mathrm{fs}]$')

for k1 in range(Nrows): axes[2*k1].set_ylabel(r'$\rho~[\mu\mathrm{m}]$')  # Set ylabel only for the left plots
for k1 in range(Nsim,2*Nrows): axes[k1].axis('off')

# Create colorbars using list comprehensions


def update(frame):
    for k1 in range(len(CUPRAD_res)):
        # Update the data for each subplot
        data = 1e-9*CUPRAD_res[k1].E_zrt[frame_multiplier*frame, :, k_t_min[k1]:k_t_max[k1]]
        
        # Update the colors
        pcs[k1].set_array(data.ravel())
        max_value_symmetric = np.max(np.abs((data.min(), data.max())))
        pcs[k1].set_clim(-max_value_symmetric,max_value_symmetric)
        cbars[k1].update_normal(pcs[k1])

        axes[k1].set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[frame_multiplier*frame]))

    return pcs

# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 

ani = matplotlib.animation.FuncAnimation(fig,
                                         update,
                                         frames=np.min([len(result.zgrid) for result in CUPRAD_res])//frame_multiplier,
                                         blit=True)

if save_animation:
    # Define the writer using ffmpeg for mp4 format and save it
    ani_outpath = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'Bessel', 'export')
    if not(os.path.exists(ani_outpath)): os.makedirs(ani_outpath)

    FFmpegWriter = matplotlib.animation.writers['ffmpeg']
    writer = FFmpegWriter(fps=fps, metadata=dict(artist=animation_author), bitrate=1800)

    ani.save(os.path.join(ani_outpath,'Bessel_pulse_propagation.mp4'), writer=writer)

HTML(ani.to_jshtml())
